In [4]:
# Standard lib imports
import os
from datetime import date
from collections import *
from glob import glob, iglob
from shutil import rmtree
import itertools
from pprint import pprint as pp

# Generic third party imports
from pycltools.pycltools import *
import pysam
import pyfaidx

# Ploting lib imports
import matplotlib.pyplot as pl
import seaborn as sns
%matplotlib inline

# Data wrangling lib imports
import pandas as pd
import numpy as np
import scipy as sp
pd.options.display.max_colwidth = 200
pd.options.display.max_columns = 200

## Make sample sheet

In [6]:
with open("./samples.tsv", "w") as fp:
    fp.write("sample_id\tcondition\treplicate\tfast5_dir\n")

    for i, fast5_dir in enumerate(sorted(glob.glob("./data/yeast/fast5/*"))):
        name = os.path.basename(fast5_dir)
        condition = name.split("_")[0]
        replicate = int(name.split("_")[1][-1])
        fast5_dir = os.path.abspath(fast5_dir)
        fp.write(f"s{i}\t{condition}\t{replicate}\t{fast5_dir}\n")

cat("./samples.tsv")

sample_id	condition	replicate	fast5_dir
s0	KO	0	/home/aleg/Programming/Packages/MetaCompore/MetaCompore/data/yeast/fast5/KO_rep0
s1	KO	1	/home/aleg/Programming/Packages/MetaCompore/MetaCompore/data/yeast/fast5/KO_rep1
s2	KO	2	/home/aleg/Programming/Packages/MetaCompore/MetaCompore/data/yeast/fast5/KO_rep2
s3	WT	0	/home/aleg/Programming/Packages/MetaCompore/MetaCompore/data/yeast/fast5/WT_rep0
s4	WT	1	/home/aleg/Programming/Packages/MetaCompore/MetaCompore/data/yeast/fast5/WT_rep1
s5	WT	2	/home/aleg/Programming/Packages/MetaCompore/MetaCompore/data/yeast/fast5/WT_rep2


## Create template files

In [21]:
for name in [
    #"input",
    "basecalling",
    "alignment",
    "quality_control",
    "quantification",
    "resquiggling",
    "nanocompore",
    "eligos2",
    "xpore",
    "tombo",
    "mines",
    "epinano",
    "differr_nanopore_DRS"]:
    
    fn = f"workflow/rules/{name}.smk"
    with open (fn, "w") as fp: 

        fp.write("""# -*- coding: utf-8 -*-

##### Imports #####

# Std lib
from os.path import join

##### Rules #####
module_name = "input"

rule_name="rule1"
rule rule1:
    input: fa=transcriptome_fa
    output: fa=join("results", module_name, rule_name, "transcriptome_reference.fa")
    log: join("logs",module_name, rule_name, "name.log")
    threads: get_threads(config, rule_name)
    params: opt=get_opt(config, rule_name)
    resources: mem_mb=get_mem(config, rule_name)
    container: "library://aleg/default/CONTAINER:TAG"
    script: "scripts/SCRIPT.py" """)



In [34]:
for name in [
    "guppy",
#     "minimap2",
#     "qc",
#     "quantification",
#     "nanopolish",
#     "nanocompore",
#     "eligos2",
#     "xpore",
#     "tombo",
#     "mines",
    #"get_transcriptome"
    ]:
    
    fn = f"workflow/scripts/{name}.py"
    with open (fn, "w") as fp: 

        fp.write("""# -*- coding: utf-8 -*-

##### Imports #####

from foo import bar


##### DEFINE SCRIPT FUNCTION #####

def function():
    pass


##### RUN SCRIPT FUNCTION #####

function():
""")

## Test functions 

In [19]:
##### Imports #####

from collections import OrderedDict
import logging
from pyBioTools import Fasta
from pyfaidx import Faidx


##### DEFINE SCRIPT FUNCTION #####

def get_transcriptome(fa_input, fa_output, fai_output, log_fn):
    with open (log_fn, "w") as log_fp:
        logging.basicConfig(filename=log_fp, level=logging.DEBUG)

        try:
            # Parse fasta file uncompress and simplify transcript ids
            logging.info("Read input transcriptome fasta file")
            with open(fa_output, "w") as fa_out:
                for rec in Fasta.Reader(fa_input):
                    fa_out.write(">{}\n{}\n".format(rec.short_name, rec.seq))

            logging.info("Index fasta file")
            with Faidx(fa_output) as fa_out:
                fa_out.build_index()

        except:
            logging.exception('Error while running get_transcriptome')
            raise

In [18]:
get_transcriptome(
    fa_input="./data/reference/transcriptome_reference.fa",
    fa_output="./results/transcriptome_reference.fa",
    fai_output="./results/transcriptome_reference.fa.fai",
    log_fn="./logs/transcriptome_reference.log")

## Downsample from fast5 source and create

In [25]:
from ont_fast5_api.fast5_interface import get_fast5_file
from ont_fast5_api.conversion_tools.fast5_subset import Fast5Filter
from random import shuffle
random.seed(42)

def get_read_id (fast5_filepath):
    l = []
    with get_fast5_file(fast5_filepath, mode="r") as f5:
        for read in f5.get_reads():
            l.append(read.read_id)            
    return l


In [31]:
n = 500
replicates = list(range())

for fast5_in in ["./data/fast5/control/FAL77200_pass_349ad0ae_0.fast5", "./data/fast5/modified/FAL82837_pass_9638a095_0.fast5"]:
    stdout_print(f"Sample source {fast5_in}\n")
    workdir = os.path.dirname(fast5_in)
    l = get_read_id(fast5_in)
    shuffle(l)
    
    s = 0
    for rep in replicates:
        stdout_print(f"\tReplicate {rep}\n")
        read_list_out = os.path.join(workdir, f"rep{rep}_read_list.txt")
        with open (read_list_out, "w") as fp:
            for r in l[s: s+n]:
                fp.write(f"{r}\n")
        s+=n
        
        stdout_print(f"\tSubsampling Reads\n")
        f = Fast5Filter(
            input_folder = workdir,
            output_folder = f"{workdir}_rep{rep}",
            read_list_file= read_list_out, 
            filename_base="batch",
            batch_size=100,
            threads=5)
        f.run_batch()
    

Sample source ./data/fast5/control/FAL77200_pass_349ad0ae_0.fast5
	Replicate 0
	Subsampling 0


DEBUG:h5py._conv:Creating converter from 5 to 3            |  0% ETA:  --:--:--
| 501 of 501|###############################################|100% Time: 0:00:01
INFO:Fast5Filter:500 reads extracted


	Replicate 1
	Subsampling 1


| 501 of 501|###############################################|100% Time: 0:00:01
INFO:Fast5Filter:500 reads extracted


	Replicate 2
	Subsampling 2


...

| 501 of 501|###############################################|100% Time: 0:00:01
INFO:Fast5Filter:500 reads extracted


	Replicate 2
	Subsampling 2


| 501 of 501|###############################################|100% Time: 0:00:01
INFO:Fast5Filter:500 reads extracted


	Replicate 3
	Subsampling 3


| 501 of 501|###############################################|100% Time: 0:00:01
INFO:Fast5Filter:500 reads extracted
